In [211]:
import os
import json
import numpy as np
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
%config InlineBackend.figure_format='retina'


subject = 'HCD0001305'
hemi = 'L'

param_name = 'DKT_MK'

mask      = f'/Users/jkember/Documents/projects/hippocampus_dMRI/mask_prob_intra.nii.gz'
surf_path = f'output/sub-{subject}_hemi-{hemi}_space-B0_den-0p5mm_label-hipp_inner.surf.gii'
vol_path  = f'output/sub-{subject}_hemi-{hemi}_space-cropB0_desc-{param_name}.nii.gz'


output_path  = f'output/sub-{subject}_hemi-{hemi}_space-cropB0_desc-{param_name}.{hemi}.func.gii'


In [ ]:

def _create_darray_GIFTI(data, hemi, map_name='None'):

    intent = nib.nifti1.intent_codes['NIFTI_INTENT_NONE']

    darray = nib.gifti.GiftiDataArray(np.array(data, dtype='float32'), intent=intent)
    darray.meta = nib.gifti.GiftiMetaData({'Name':map_name})

    if hemi == 'L': meta = nib.gifti.GiftiMetaData({'AnatomicalStructurePrimary':'HippocampusLeft'})
    if hemi == 'R': meta = nib.gifti.GiftiMetaData({'AnatomicalStructurePrimary':'HippocampusRight'})

    gii = nib.GiftiImage(darrays=[darray], meta=meta)

    return gii



'''
    mask: Path to NIFTI with partial-volume data (0-1: prob. intra-hippocampal).

    '''
radius = 3
sigma_mm = .5



# Load data data.
mask_nii = nib.load(mask)
mask_data = mask_nii.get_fdata()
mask_data_flat = mask_data[mask_data > 0]

vol_nii    = nib.load(vol_path)
vol_data   = vol_nii.get_fdata()
vol_affine = vol_nii.affine

surf       = nib.load(surf_path)
vertex_mm  = surf.darrays[1].data
n_vertices = len(vertex_mm)


# Get coordinates of masked voxels.
voxel_indices = np.column_stack(np.where(mask_data > 0))
voxel_mm = nib.affines.apply_affine(vol_affine, voxel_indices)

# Get masked-voxel data.
n_voxels = len(voxel_indices)
voxel_data = np.full(n_voxels, np.nan)

for idx, voxel_idx in enumerate(voxel_indices):
    x, y, z = voxel_idx
    voxel_data[idx] = vol_data[x, y, z]


# Map voxel data to surface-vertex, weighted by distance and partial-volume.
voxel_tree = cKDTree(voxel_mm)

vertex_values = np.full(n_vertices, np.nan)
for v_idx, vertex in enumerate(vertex_mm):

    nearby_vox = voxel_tree.query_ball_point(vertex,r=radius)

    nearby_vox_coords = voxel_mm[nearby_vox]
    distances = np.linalg.norm(nearby_vox_coords - vertex, axis=1)

    # Voxels weighted by partial-volume and gaussian-weighted distance.
    distance_W = np.exp(-0.5 * (distances / sigma_mm)**2)
    partial_volume_W = mask_data_flat[nearby_vox]
    W = distance_W * partial_volume_W

    vertex_values[v_idx] = np.average(voxel_data[nearby_vox], weights=W)


gii = _create_darray_GIFTI(vertex_values, hemi, map_name=param_name)
nib.save(gii, output_path)

